View Fit Result on A single Spectrum
------------------------------------


Analyse outputs from GELATO fit results on a signe spectrum

- author : Sylvie Dagoret-Campagne
- creation date : 2024-03-25
- update : 2024-04-17


- Kernel at CCIN2P3 : ``conda_desc_py310_pcigale``


In [ ]:
# Import packages
import gelato
import numpy as np
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['font.size'] = 25
from matplotlib import pyplot # For plotting
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
# For loading in data
from astropy.io import fits
from astropy.table import Table 
import os
import re
from collections import OrderedDict
import pandas as pd

In [ ]:
#from gelato.Plotting import  Plot, PlotFig,subplotplot
#from gelato.Plotting import subplotplot
from gelato.Plotting import logbarrier
from scipy.optimize import minimize

#import gelato.ConstructParams as CP

import gelato.Utility as U
import gelato.Plotting as P
import gelato.ConstructParams as CP

# GELATO
import gelato.Utility as U
import gelato.CustomModels as CM
import gelato.SpectrumClass as SC


### redefined functions of GELATO to View in notebook

In [ ]:
# Add Linelabels
def Mysubplotplot(pt,fax,rax,hax,spectrum,args,f):
    
    # Unpack
    wav,flux,isig = args

    # Plot data
    fax.step(wav,flux,'gray',where='mid')

    # Plot model(s)
    # for p in parameters:
    #     fax.step(wav,model.evaluate(p,*args),'r',where='mid',alpha=0.5)
    fax.step(wav,f,'r',where='mid')
    
    # Base Y axis on flux
    ymin = np.max([0,flux.min()])
    dy = flux.max() - ymin
    ylim = [ymin,ymin+1.3*dy] # Increase Axis size by 20%
    text_height = ymin+1.2*dy 

    # Get Line Names/Positions
    linelocs = []
    linelabels = []
    for group in spectrum.p['EmissionGroups']:
        for species in group['Species']:
            if species['Flag'] >= 0:
                for line in species['Lines']:
                    x = line['Wavelength']*(1+spectrum.z)
                    if ((x < wav[-1]) and (x > wav[0])):
                        linelocs.append(x)
                        linelabels.append(species['Name'])

    # If we have lines to plot
    if len(linelocs) > 0:

        # Reorder line positions
        inds = np.argsort(linelocs)
        linelocs = np.array(linelocs)[inds]
        linelabels = np.array(linelabels)[inds]

        # Log barrier constraints
        norm = 60 if pt == 0 else 20 # Magic numbers for line spacing
        x0 = np.linspace(wav.min(),wav.max(),len(inds)+2)[1:-1] # Initial guess
        linelabellocs = minimize(logbarrier,x0,args=(wav,linelocs,norm),method='Nelder-Mead',options={'adaptive':True,'maxiter':len(inds)*750}).x

        # Plot names
        for lineloc,linelabel,linelabelloc in zip(linelocs,linelabels,linelabellocs):
            # Text
            fax.text(linelabelloc,text_height,linelabel,rotation=90,fontsize=12,ha='center',va='center')
            # Plot Lines
            fax.plot([lineloc,lineloc,linelabelloc,linelabelloc],[ymin+dy*1.01,ymin+dy*1.055,ymin+dy*1.075,ymin+dy*1.12],ls='-',c='gray',lw=0.25)

    # Axis labels and limits
    fax.set(ylabel=r'$F_\lambda$ ['+spectrum.p['FlamUnits']+']',ylim=ylim)
    fax.set(yticks=[t for t in fax.get_yticks() if (t > ymin+0.05*dy) and (t < ylim[-1])],xlim=[wav.min(),wav.max()],xticklabels=[])
    
    
   
    
    # Residual Axis
    
    pulls = (flux - f)*isig
    p_m = np.mean(pulls)
    p_s = np.std(pulls)
    p_l = f"pulls = {p_m:.1f} +/- {p_s:.1f}"
    
    rax.step(wav,pulls,'gray',where='mid')
    ymax = np.max(np.abs(rax.get_ylim()))
    rax.set(xlim=[wav.min(),wav.max()],xlabel=r'Observed Wavelength [\AA]',ylim=[-ymax,ymax])
    rax.set_ylabel('Deviation',fontsize=15)
    
    if pt == 0:
        inset_rax = inset_axes(rax,
                    width="20%", # width = 30% of parent_bbox
                    height="80%", # height : 1 inch
                    loc=4)
        inset_rax.hist(pulls,bins=30,range=(-5,5),density=True, histtype='step', facecolor='g',lw=2,color="g",label=p_l)
        inset_rax.set_xlabel("pulls")
        inset_rax.legend(loc="upper right",prop={'size': 16})
        
    
    elif pt>0 and hax!= None:
        
        hax.hist(pulls,bins=30,range=(-5,5),density=True, histtype='step', facecolor='g',lw=2,color="g",label=p_l)
        hax.set_xlabel("pulls")
        hax.legend(loc="upper right",prop={'size': 16})
    return ymin

In [ ]:
# Overwrite functions from GELATO to view results in a notebook

from matplotlib import rcParams,pyplot
def MyPlotFig(spectrum,model,parameters,fpath,plottype=0):

    # Calculate Medians
    medians = np.median(parameters,0)

    # Make figure name
    #figname = U.fileName(path.split(fpath)[-1])+'-'  OLD
    figname= U.fileName(fullfilename.split("/")[-1].split(".")[0])+'-'  #SDC
    
    if plottype == 0:
        figname += 'spec'
    elif plottype == 1:
        figname += 'fit'
    elif plottype == 2:
        figname += 'comp'

  
    # Get transform for secondary axis
    transform = (lambda obs: obs / (1 + spectrum.z), lambda rest: rest * (1 + spectrum.z))

    if plottype == 0:

        # Make figure
        fig = pyplot.figure(figsize=(15,10))
        gs = fig.add_gridspec(ncols=1,nrows=2,height_ratios=[4,1],hspace=0)

        # Get Spectrum
        wav     = spectrum.wav
        flux    = spectrum.flux
        isig    = spectrum.isig

        # Model prediction
        args = wav,flux,isig
        f = model.evaluate(medians,*args)

        # Add axes flux axis and residuals axis
        fax,rax = fig.add_subplot(gs[0,0]),fig.add_subplot(gs[1,0])

        # Plot Power Law
        if 'PowerLaw_Index' in model.get_names():
            continuum = CM.CompoundModel(model.models[1:2]).evaluate(medians[model.models[0].nparams:],*args)
            fax.step(wav,continuum,'k',ls='--',where='mid')

        # Plot model(s)
        #for p in parameters:
        #     fax.step(wav,model.evaluate(p,*args),'r',where='mid',alpha=0.5)
        fax.step(wav,f,'r',where='mid')

        # Subplot plotting
        hax=None
        Mysubplotplot(plottype,fax,rax,hax,spectrum,args,f)

        # Add secondary axis
        rax.secondary_xaxis('top', functions=transform).set(xticklabels=[])
        fax.secondary_xaxis('top', functions=transform).set_xlabel('Rest Wavelength [\AA]',labelpad=10)
        rax.axhline(0,color="b")
            
    elif plottype > 0:

        # Make figure
        ncols   = len(spectrum.regions)
        fig = pyplot.figure(figsize = (5*ncols,10))
        gs = fig.add_gridspec(ncols=ncols,nrows=3,height_ratios=[3,1,2],hspace=0)

        # Continuum and Model
        args = spectrum.wav,spectrum.flux,spectrum.isig
        if 'PowerLaw_Index' in model.get_names():
            continuum = CM.CompoundModel(model.models[0:2]).evaluate(medians,*args)
        else: 
            continuum = CM.CompoundModel(model.models[0:1]).evaluate(medians,*args)
        f = model.evaluate(medians,*args)
        
        # Iterate over regions
        for i,region in enumerate(spectrum.regions):

            # Get Spectrum
            good    = np.logical_and(spectrum.wav < region[1],spectrum.wav > region[0])
            wav     = spectrum.wav[good]
            flux    = spectrum.flux[good]
            isig    = spectrum.isig[good]
            args    = wav,flux,isig

            # Add Axes
            fax,rax,hax = fig.add_subplot(gs[0,i]),fig.add_subplot(gs[1,i]),fig.add_subplot(gs[2,i])

            # Subplot plotting
            ymin = Mysubplotplot(plottype,fax,rax,hax,spectrum,args,f[good])

            # Plot Continuum
            if plottype == 1:
                fax.step(wav,continuum[good],ls='-',c='k',where='mid')
            # Plot components
            elif plottype == 2:
                init = 1
                if 'PowerLaw_Index' in model.get_names():
                    init = 2
                for j in range(init,len(model.models)):
                    m = model.models[j]
                    idx = model.indices[j]
                    cm = CM.CompoundModel([m]).evaluate(medians[idx:idx+m.nparams],*(wav,flux,isig))
                    fax.step(wav,ymin+cm,'--',c='gray')
                    # for p in parameters:
                    #     cm = CM.CompoundModel([m]).evaluate(p[idx:idx+m.nparams],*(wav,flux,isig))
                    #     fax.step(wav,ymin+cm,'--',c='gray',alpha=0.5)

            # Add secondary axis
            rax.secondary_xaxis('top', functions=transform).set(xticklabels=[])
            fax.secondary_xaxis('top', functions=transform).set_xlabel('Rest Wavelength [\AA]',labelpad=10)
            rax.axhline(0,color="b")
            
    # Add title and save figure
    fig.suptitle(figname.replace('_','\_')+', $z='+str(np.round(spectrum.z,3))+'$',y=0.95)
    fig.tight_layout()
    #fig.savefig(path.join(spectrum.p['OutFolder'],figname+'.pdf'))
    plt.show()
    #pyplot.close(fig)


In [ ]:
# Plot all Figures
def MyPlot(spectrum,model,parameters,fpath):
    for i in range(3): MyPlotFig(spectrum,model,parameters,fpath,plottype=i)

In [ ]:
# Plot from results
def myplotfromresults(params,fpath,z):

#    if params["Verbose"]:
#        print("Presenting GELATO:",path.split(fpath)[-1])

    ## Load in Spectrum ##
  
    spectrum = SC.Spectrum(fpath,z,params)
    
    #------------------------
    #print("spectrum",spectrum)
    #--------------------------
  
    # Get just the final bit of the path
    #fpath = path.split(fpath)[-1] #SDC remove this split
    #print(fpath)
    
    ## Load Results ##
    #fname = path.join(params['OutFolder'],U.fileName(fpath))+'-results.fits'
    fname = fpath
    parameters = fits.getdata(fname,'PARAMS')
    pnames =  [n for n in parameters.columns.names if not (('EW' in n) or ('RAmp' in n) or ('PowerLaw_Scale' == n))][:-1]
    ps = np.array([parameters[n] for n in pnames]).T
    
    #-------------
    # print(pnames)
    # print("ps",ps)
    #-------------

    ## Create model ##
    # Add continuum
    ssp_names = [n[4:] for n in pnames if (('SSP_' in n) and (n != 'SSP_Redshift'))]
    
    #----------------------------
    #print("(ssp_names",ssp_names)
    #----------------------------
    
    
    models = [CM.SSPContinuumFree(spectrum,ssp_names = ssp_names)]
    if 'PowerLaw_Index' in pnames:
        models.append(CM.PowerLawContinuum(spectrum))
        models[-1].starting()
    
    #-----------------------------------------
    #print("spectrum.regions",spectrum.regions)
    #-----------------------------------------

    #if spectrum.regions != []:
    if len(spectrum.regions) != 0:

        # Add spectral lines
        ind = sum([m.nparams for m in models]) # index where emission lines begin
        for i in range(ind,ps.shape[1],3):
            center = float(pnames[i].split('_')[-2])
            models.append(CM.SpectralFeature(center,spectrum))

        # Final model
        model = CM.CompoundModel(models)
        #print("final model",model)

        # Plot
        MyPlot(spectrum,model,ps,fpath)

    else:

        # Final Model
        model = CM.CompoundModel(models)

        # Plot
        MyPlotFig(spectrum,model,ps,fpath)

    if params["Verbose"]:
        print("GELATO presented:",fpath)

In [ ]:
def MySimplePlotSpectrum(fullfilename ,redshift,title):
    """
    My simple plot spectrum without model
    """
    # Let's load the spectrum
    #path_spec = 'Spectra/spec-0280-51612-0117.fits'
    path_spec = fullfilename 
    spectrum = Table.read(path_spec)

    # Start with inverse variance
    ivar = spectrum['ivar']
    good = ivar > 0 # GELATO only looks at points with nonzero weights

    # Finally, let's load in the data
    wavl = 10**spectrum['loglam'][good]
    flux = spectrum['flux'][good]
    ivar = ivar[good]
    args = (wavl,flux,ivar) # These will be useful later
        
    # Create figure
    fig, ax = pyplot.subplots(figsize=(15,7))

    # Plot Spectrum
    sig = 3/np.sqrt(ivar) # 3 Sigma boundary
    ax.fill_between(wavl,flux-sig,flux+sig,color='gray')
    ax.step(wavl,flux,where='mid',c='k',lw=0.5)

    # Axis limits
    ax.set(xlim=[wavl.min(),wavl.max()],ylim=[0,flux.max()])

    # Axis labels
    ax.set(xlabel=r'Obs. Wavelength [\AA]',ylabel=r'$F_\lambda$')

    ax.set_title(title)
    # Show figure
    pyplot.show()
    

In [ ]:
def MySimplePlotSpectrumWithFittedModel(fullfilename ,redshift,title):
    """
    My simple plot spectrum with fitted model
    """
    # Let's load the spectrum
    #path_spec = 'Spectra/spec-0280-51612-0117.fits'
    path_spec = fullfilename 
    spectrum = Table.read(path_spec)
    
    # loat fit results
    results = fits.open(path_spec) 
    summary = Table(results['SUMMARY'].data)

    # Start with inverse variance
    ivar = spectrum['ivar']
    good = ivar > 0 # GELATO only looks at points with nonzero weights

    # Finally, let's load in the data
    wavl = 10**spectrum['loglam'][good]
    flux = spectrum['flux'][good]
    ivar = ivar[good]
    args = (wavl,flux,ivar) # These will be useful later
        
    # Plot Spectrum
    sig = 3/np.sqrt(ivar) # 3 Sigma boundary
    
        
    # Create figure
    fig, ax = pyplot.subplots(figsize=(15,7))

    
    # Plot Spectrum
    ax.fill_between(wavl,flux-sig,flux+sig,color='gray',alpha=0.2)
    ax.step(wavl,flux,where='mid',c='k',lw=0.75,label='Data')
    ax.step(10**summary['loglam'],summary['MODEL'],where='mid',c='r',label='Total Model')
    ax.step(10**summary['loglam'],summary['SSP'],where='mid',c='g',label='SSP Cont.')
    #ax.step(10**summary['loglam'],summary['PL'],where='mid',c='b',label='Power-Law Cont.')
    ax.step(10**summary['loglam'],summary['LINE'],where='mid',c='orange',lw=3,label='Emission Lines')
    ax.legend(loc="upper left")

    # Axis limits
    ax.set(xlim=[wavl.min(),wavl.max()],ylim=[0,flux.max()*1.5])

    # Axis labels
    ax.set(xlabel=r'Obs. Wavelength [\AA]',ylabel=r'$F_\lambda$')

    ax.set_title(title)
    # Show figure
    pyplot.show()
    

## Config

### Table with spectra name / redshifts

In [ ]:
df = pd.read_csv("object_filelist_v0.csv",index_col=0)

In [ ]:
df

### Gelato Input parameters

In [ ]:
# Path to the parameters file
path_params = './ExampleParameters.json'

In [ ]:
params_gel = CP.construct(path_params)

In [ ]:
params_gel

In [ ]:
path = "Results_v0"

In [ ]:
params_gel['OutFolder'] = path

In [ ]:
list_all_files = os.listdir(path)

In [ ]:
list_all_files[:5]

In [ ]:
re.findall(".*[.]fits$",'specgelato_SPEC308-results.fits')

In [ ]:
int(re.findall("specgelato_SPEC(.*)-results[.]fits$",'specgelato_SPEC308-results.fits')[0])

In [ ]:
idx_selected_files = []
list_selected_files = []
for file in list_all_files:
    res = re.findall("^specgelato_SPEC.*[.]fits$",file)
    if len(res):
        list_selected_files.append(file)
        num = int(re.findall("specgelato_SPEC(.*)-results[.]fits$",file)[0])   
        idx_selected_files.append(num)

In [ ]:
idx_selected_files = np.array(idx_selected_files)
list_selected_files = np.array(list_selected_files)
idx_sorted_files = np.argsort(idx_selected_files)
list_sorted_files = list_selected_files[idx_sorted_files]

## Select one Spectrum

In [ ]:
idx=0

In [ ]:
shortfilename = list_sorted_files[idx]
fullfilename = os.path.join(path,list_sorted_files[idx])
title = f"{idx}) {fullfilename}" 

In [ ]:
print(shortfilename,fullfilename)

In [ ]:
tag_spec = re.findall(".*_(SPEC.*)-results.fits$", shortfilename)

In [ ]:
if len(tag_spec)>0:
    tag_spec = tag_spec[0]

In [ ]:
tag_spec

In [ ]:
all_inputspecfilenames = df.Path.values
for idx_tag,filen in enumerate(all_inputspecfilenames):
    if tag_spec in filen:
        break
idx_tag
df_row = df.iloc[idx_tag]
redshift = df_row["z"]
title = title + f" z ={redshift:.3f}"

In [ ]:
path_spec = fullfilename

In [ ]:
MySimplePlotSpectrum(fullfilename ,redshift,title)

The main gelato function takes three inputs.
* The path to the parameters file or the parameters dictionary.
* The path to the spectrum.
* The redshift of the spectrum.

We already have the last two, and we need to take a little precaution with the first.
The main gelato function will only return the final model if the code is being run without multiprocessing (as the return statement can break Python multiprocessing). So we can either change the Parameters JSON file, or edit the parameters dictionary. 

In [ ]:
# Path to the parameters file
#path_params = './ExampleParameters.json'

# Create Parameters dictionary
#params = gelato.ConstructParams.construct(path_params)

# Set to not multiprocessing
#params['NProcess'] = 1

We are now ready to run GELATO. Note, before you do this, ensure the results directory exists, either by running the Example from the README file or creating it. It will return the final callable model, however it won't be used in this notebook. 

In [ ]:
#model = gelato.gelato(params,path_spec,0.69)

The results have been saved to the "Results/" Directory. Let's go ahead and load them in. We will print all extensions on the folder.

In [ ]:
path_spec

In [ ]:
# Load in results
results = fits.open(path_spec)

# Print FITS extensions
results.info()

In [ ]:
header = Table(results['PRIMARY'].data)

In [ ]:
header

We have two FITS extensions, SUMMARY and PARAMS. They are described in more detail in the README File but let's play around with them directly. Let's go ahead and take a look inside the SUMMARY extension. As we can see, it is a binary FITS Table.

In [ ]:
summary = Table(results['SUMMARY'].data)
summary

In [ ]:
MySimplePlotSpectrumWithFittedModel(fullfilename ,redshift,title)

In this table, we have the original spectrum along with the various model components, we can go ahead and plot them.

Looks great! You can see an example of the GELATO generated plots in the results folder, but this will let you incorporate GELATO fits easily into your own work. Let's go ahead and take a look at the PARAMS extension. This is a much larger table! It's made up of the parameters from each bootstrap iteration. 

Here we can see all of the fitted model paramters, it's certainly a handful! A quick note, many parameters here are tied together, reducing the degrees of freedom. It's also worth noting the SSP Continuum Redshift and the PL Continuum Scale are not fitted, and so are constant throughout all the bootstraps. Let's go ahead and throw this object onto a BPT diagram!

## Call gelato plot fucntion

In [ ]:
# Ceci marche
# parameters = fits.getdata(fullfilename,'PARAMS')

In [ ]:
fullfilename.split("/")[-1].split(".")[0]

In [ ]:
U.fileName(fullfilename.split("/")[-1].split(".")[0])+'-'

In [ ]:
U.fileName(path.split(fullfilename)[-1])+'-'

In [ ]:
path.split(fullfilename)

In [ ]:
U.fileName(path.split(fullfilename))

In [ ]:
U.fileName

In [ ]:
#Plot(spectrum,results,path_params,fpath=path_spec)
#myplotfromresults(path_params, fullfilename, redshift)
myplotfromresults(params_gel,fullfilename, redshift)

And that's all for this short IPython notebook! GELATO is designed to be run through the wrapper scripts, but hopefully this helps if you want to run in an IPython notebook or to help you understand the GELATO output.

In [ ]:
results['PARAMS'].columns

In [ ]:
# Open Parameters extension
params = Table(results['PARAMS'].data)
print(params)


In [ ]:
for col in params.colnames:
    print(col)

Here we can see all of the fitted model paramters, it's certainly a handful! A quick note, many parameters here are tied together, reducing the degrees of freedom. It's also worth noting the SSP Continuum Redshift and the PL Continuum Scale are not fitted, and so are constant throughout all the bootstraps. Let's go ahead and throw this object onto a BPT diagram!

In [ ]:
selected_col_Flux = []
for col in params.columns:
    if "Flux" in col:
        selected_col_Flux.append(col)
        
    

In [ ]:
params_Flux = params[selected_col_Flux]

In [ ]:
params_Flux.to_pandas().describe()

In [ ]:
plt.plot(params["rChi2"])

In [ ]:
all_ser = []
for idx,file in enumerate(list_sorted_files):   
    path_spec =  os.path.join(path,file)
    results = fits.open(path_spec)
    #spectrum = Table.read(path_spec)
    params = Table(results['PARAMS'].data)
    selected_col_Flux = []
    for col in params.columns:
        if "Flux" in col:
            selected_col_Flux.append(col)
    selected_col = selected_col_Flux 
    selected_col.append("rChi2")
    t_sub = params[selected_col]
    df = t_sub.to_pandas()
    ser = df.median()
    #print(ser)
    all_ser.append(ser)
    

In [ ]:
pd.concat(all_ser, axis=1).T